In [1]:
import pandas as pd
import time
import os, gc
import numpy as np
import re

In [2]:
from sklearn.metrics import roc_auc_score

In [3]:
%%time
#train_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_train.csv")
#test_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_test.csv")
train_df=pd.read_csv("../KB_NLP/jamo_data/jamo_train.csv")
test_df=pd.read_csv("../KB_NLP/jamo_data/jamo_test.csv")

CPU times: user 2.76 s, sys: 185 ms, total: 2.94 s
Wall time: 4.78 s


In [4]:
train_df.head()

,id,year_month,text,smishing,jamo
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,ㅇㅡㄴㅎㅐㅇㅅㅓㅇㅅㅏㄴ ㅌㅣㅁㅈㅏㅇㅇㅣㅂㄴㅣㄷㅏ ㅎㅐㅇㅂㅗㄱㅎㅏㄴㅈㅜㅁㅏㄹㄷㅚㅅㅔㅇㅛ
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,ㅇㅗㄴㅡㄹㄷㅗㅁㅏㄶㅇㅣㅇㅜㅅㅇㅡㅅㅣㄴㅡㄴㅎㅏㄹㅜㅅㅣㅈㅏㄱㅎㅏㅅㅔㅇㅛ ㅇㅡㄴㅎㅐㅇ ㅈ...
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ ㄱㅗㄱㅐㄱㄴㅣㅁ ㅇㅡㄴㅎㅐㅇㅇㅣㅂㄴㅣㄷㅏ ㄱㅡㅁㅇㅣㄹ ...
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,ㄱㅗㄱㅐㄱㄴㅣㅁㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ ㅇㅡㄴㅎㅐㅇ ㅈㅣㅈㅓㅁㅇㅣㅂㄴㅣㄷㅏㅈㅣㄴㅏㄴ...
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,ㅇㅝㄹㅇㅡㄴ ㅅㅐㄹㅗㅇㅜㅁㅇㅣ ㄱㅏㄷㅡㄱ ㅇㅣㅂㄴㅣㄷㅏ ㅇㅗㄹ ㅎㅏㄴㅎㅐ ㄷㅓ ㅁ...


In [5]:
%%time
with open("../KB_NLP/jamo_label_train.txt",'w') as f:
    for i in range(len(train_df)): #
        f.write("__label__")
        f.write("{}".format(train_df['smishing'][i]))
        f.write(" {}".format(train_df['jamo'][i]))
        f.write("\n")

print("Make Labeled jamo")

Make Labeled jamo
CPU times: user 4.29 s, sys: 227 ms, total: 4.51 s
Wall time: 7.01 s


In [6]:
f = open('../KB_NLP/jamo_label_train.txt','r')
train_text = f.readlines()
f.close()

In [7]:
train_text[:4]

['__label__0  ㅇㅡㄴㅎㅐㅇㅅㅓㅇㅅㅏㄴ ㅌㅣㅁㅈㅏㅇㅇㅣㅂㄴㅣㄷㅏ ㅎㅐㅇㅂㅗㄱㅎㅏㄴㅈㅜㅁㅏㄹㄷㅚㅅㅔㅇㅛ\n',
 '__label__0 ㅇㅗㄴㅡㄹㄷㅗㅁㅏㄶㅇㅣㅇㅜㅅㅇㅡㅅㅣㄴㅡㄴㅎㅏㄹㅜㅅㅣㅈㅏㄱㅎㅏㅅㅔㅇㅛ ㅇㅡㄴㅎㅐㅇ ㅈㅣㄴㅇㅝㄹㄷㅗㅇ ㄹㅏㅇㅜㄴㅈㅣ ㅇㅗㄹㄹㅣㅁ\n',
 '__label__0 ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅣㅂㄴㅣㄲㅏ ㄱㅗㄱㅐㄱㄴㅣㅁ ㅇㅡㄴㅎㅐㅇㅇㅣㅂㄴㅣㄷㅏ ㄱㅡㅁㅇㅣㄹ ㄴㅏㅂㅂㅜㅎㅏㅅㅕㅇㅑ ㅎㅏㄹ ㄱㅡㅁㅇㅐㄱㅇㅡㄴ ㅇㅝㄴ ㅇㅣㅂㄴㅣㄷㅏ ㄱㅏㅁㅅㅏㅎㅏㅂㄴㅣㄷㅏ ㅅㅐㅎㅐ ㅂㅗㄱ ㅁㅏㄶㅇㅣ ㅂㅏㄷㅇㅡㅅㅣㅂㅅㅣㅇㅗ ㅇㅡㄴㅎㅐㅇㅇㅗㄱㅍㅗ ㅇㅗㄹㄹㅣㅁ\n',
 '__label__0  ㄱㅗㄱㅐㄱㄴㅣㅁㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ ㅇㅡㄴㅎㅐㅇ ㅈㅣㅈㅓㅁㅇㅣㅂㄴㅣㄷㅏㅈㅣㄴㅏㄴ ㅎㅏㄴ ㅎㅐ ㄷㅗㅇㅇㅏㄴ ㅈㅓㅎㅢ ㅈㅣㅈㅓㅁㅇㅔ ㅂㅗㄴㅐㅈㅜㅅㅣㄴ ㅅㅓㅇㅇㅝㄴㅇㅔ ㄱㅏㅁㅅㅏㄷㅡㄹㅣㅂㄴㅣㄷㅏ ㅅㅓㄹㄹㅔㅁㅇㅡㄹㅗ ㅅㅣㅈㅏㄱㅎㅏㄴ ㄴㅕㄴㅅㅗㅁㅏㅇㅎㅏㅅㅣㄴㅡㄴ ㅇㅣㄹ ㅁㅗㄷㅜ ㅇㅣ ㄱㅗㄱㅐㄱㄴㅣㅁㅇㅢ ㄱㅏㅈㅓㅇㅇㅔ ㄴㅡㄹ ㄱㅓㄴㄱㅏㅇㄱㅘ ㅎㅐㅇㅂㅗㄱㅇㅣ ㅎㅏㅁㄲㅔㅎㅏㄱㅣㄹ ㄱㅣㅇㅝㄴㅎㅏㄱㅔㅆㅅㅡㅂㄴㅣㄷㅏ ㅅㅏㄹㅏㅇㅎㅏㄴㅡㄴ ㄱㅏㅈㅗㄱㄱㅘ ㅎㅏㅁㄲㅔ ㅈㅓㅇㅇㅡㄹ ㄴㅏㄴㅜㄴㅡㄴ ㅎㅐㅇㅂㅗㄱㅎㅏㄴ ㅅㅓㄹ ㅁㅕㅇㅈㅓㄹ ㅂㅗㄴㅐㅅㅔㅇㅛ ㅇㅡㄴㅎㅐㅇ ㅈㅣㅈㅓㅁㅈㅣㄱㅇㅝㄴㅇㅣㄹㄷㅗㅇ\n']

In [8]:
import csv

train_text = pd.DataFrame(train_text)
train_text.to_csv("../KB_NLP/jamo_label_train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar='', escapechar=' ')

In [9]:
import fasttext

In [10]:
model = fasttext.train_supervised("../KB_NLP/jamo_label_train.txt", label_prefix='__label__', thread=2, epoch=10)
print(model.labels, "labels for predicting")

['__label__0', '__label__1'] labels for predicting


---
### Prediction

In [12]:
test_df['jamo'][0]

' ㄱㅗㄱㅐㄱㄴㅣㅁ ㅇㅏㄴㄴㅕㅇㅎㅏㅅㅔㅇㅛ ㅅㅐㄹㅗㅂㄱㅔ ㅅㅣㅈㅏㄱㅎㅏㄴㅡㄴ ㅎㅏㄴ ㅈㅜ ㅎㅐㅇㅂㅗㄱ ㄱㅏㄷㅡㄱㅎㅏㅅㅣㄱㅣㄹ ㄱㅣㅇㅝㄴㅎㅏㅂㄴㅣㄷㅏ ㅈㅣㄴㅏㄴㅈㅜ ㄱㅡㄹㄹㅗㅂㅓㄹ ㅈㅡㅇㅅㅣㄴㅡㄴ ㅍㅏㅇㅝㄹ ㅁㅣ ㅇㅕㄴㅈㅜㄴ ㅇㅢㅈㅏㅇㅇㅣ ㅌㅗㅇㅎㅘㅈㅓㅇㅊㅐㄱㅇㅔ ㄷㅐㅎㅐ ㅇㅘㄴㅎㅘㅈㅓㄱㅇㅣㄴ ㅌㅐㄷㅗ ㅂㅕㄴㅎㅘㄹㅡㄹ ㅂㅗㅇㅣㄴ ㄱㅓㅅㅇㅣ ㅈㅜㅊㅗ ㅌㅜㅈㅏㅅㅣㅁㄹㅣ ㄱㅐㅅㅓㄴㅇㅡㄹㅗ ㅇㅣㅇㅓㅈㅣㅁㅕ ㅋㅡㄴ ㅍㅗㄱ ㅅㅏㅇㅅㅡㅇ ㅁㅏㄱㅏㅁㅎㅐㅆㅅㅡㅂㄴㅣㄷㅏ ㄷㅓㅂㅜㄹㅇㅓ ㅁㅣㅈㅜㅇ ㅊㅏㄱㅘㄴㄱㅡㅂ ㅁㅜㅇㅕㄱㅎㅕㅂㅅㅏㅇㅇㅣ ㅅㅜㄴㅈㅗㄹㅗㅂㄱㅔ ㅈㅣㄴㅎㅐㅇㄷㅚㄴ ㅈㅓㅁㄷㅗ ㄱㅡㅇㅈㅓㅇㅈㅓㄱㅇㅣㅇㅓㅆㅅㅡㅂㄴㅣㄷㅏ ㅇㅝㄴㄷㅏㄹㄹㅓ ㅎㅘㄴㅇㅠㄹㅇㅣ ㅈㅜㄱㅏㄴ ㅇㅝㄴㅇㅡㄹㅗ ㅎㅏㄹㅏㄱㅎㅏㄴㅡㄴ ㄱㅏㅇㅜㄴㄷㅔ ㅇㅚㄱㅜㄱㅇㅣㄴ ㅌㅜㅈㅏㅈㅏㅇㅢ ㄱㅜㄱㄴㅐㅈㅡㅇㅅㅣ ㅅㅜㄴㅁㅐㅅㅜ ㄱㅠㅁㅗㄱㅏ ㅊㅓㄴㅇㅓㄱㅇㅡㄹ ㅅㅏㅇㅎㅚㅎㅏㅁㅕ ㅅㅜㄱㅡㅂㅇㅣ ㄱㅐㅅㅓㄴㄷㅚㄴ ㅈㅓㅁㄷㅗ ㅈㅜㅁㅗㄱㅎㅐㅂㅗㄹ ㅍㅣㄹㅇㅛㄱㅏ ㅇㅣㅆㅅㅡㅂㄴㅣㄷㅏ ㄱㅡㅁㅈㅜ ㅅㅣㅈㅏㅇㅇㅡㄴ ㅁㅣㄱㅜㄱ ㄱㅡㅁㅇㅠㅇㅈㅜ ㅅㅣㄹㅈㅓㄱㅂㅏㄹㅍㅛㄹㅡㄹ ㅅㅣㅈㅏㄱㅇㅡㄹㅗ ㅂㅗㄴㄱㅕㄱㅈㅓㄱㅇㅣㄴ ㅅㅣㄹㅈㅓㄱ ㅅㅣㅈㅡㄴㅇㅔ ㅈㅣㄴㅇㅣㅂㅎㅏㄱㅔ ㄷㅚㅂㄴㅣㄷㅏ ㅈㅣㄴㅏㄴㅈㅜ ㅇㅐㅍㅡㄹ ㅅㅏㅁㅅㅓㅇㅈㅓㄴㅈㅏ ㄷㅡㅇ ㅈㅜㅇㅛㄱㅣㅇㅓㅂㄷㅡㄹㅇㅢ ㅂㅜㄴㄱㅣ ㅅㅣㄹㅈㅓㄱ ㅈㅓㄴㅁㅏㅇ ㅎㅏㅎㅑㅇ ㅈㅗㅈㅓㅇㅇㅣ ㅇㅣㅆㅇㅓㅆㄷㅓㄴ ㅈㅓㅁㅇㅡㄹ ㅊㅏㅁㄱㅗㅎㅏㄴㄷㅏㅁㅕㄴ ㅇㅗㄹㅎㅐ ㄱㅡㄹㄹㅗㅂㅓㄹ ㅅㅓㅇㅈㅏㅇ ㄷㅜㄴㅎㅘ ㄱㅣㅈㅗㅅㅗㄱㅇㅔㅅㅓ ㄱㅣㅇㅓㅂㄷㅡㄹㅇㅢ ㅅㅜㄴㅇㅣㄱ ㄱㅏㅁㅅㅗ ㅊㅜㅇㅣㄹㅡㄹ ㄱㅘㄴㅅㅣㅁㅇㅣㅆㄱㅔ ㅅㅏㄹㅍㅕㅂㅗㅅㅣㄹ ㅍㅣㄹㅇㅛㄱㅏ ㅇㅣㅆㅇㅓ ㅂㅗㅇㅣㅂㄴㅣㄷㅏ ㅈㅜ ㅎㅜㅂㅏㄴㅇㅔㄴㅡㄴ ㅂㅡㄹㅔㄱㅅㅣㅌㅡ ㅇㅕㅇㄱㅜㄱ ㅎㅏㅇㅝㄴ ㅍㅛㄱㅕㄹㅇㅣ ㅇㅖㅈㅓㅇㄷㅚㅇㅓ ㅇㅣㅆㅅㅡㅂㄴㅣㄷㅏ ㅊㅜㅇㅝㅈㅣㄴ ㄴㅏㄹㅆㅣㅇㅔ ㄱㅓㄴㄱㅏㅇ ㅇㅠㅇㅢㅎㅏㅅㅣㄱㅗ ㅈㅗㅎㅇㅡㄴ ㅎㅏㄴㅈㅜ ㄷㅚㅅㅔㅇㅛ ㄱㅏㅁㅅㅏㅎㅏㅂㄴㅣㄷㅏ ㅇㅡㄴㅎㅐㅇㅅㅗㅇㄷㅗ ㅅㅔㄴㅌㅓ ㅇㅗㄹㄹㅣㅁ'

In [18]:
from tqdm import tqdm

In [19]:
prob_list = []
for i in tqdm(range(len(test_df))):
    pred = model.predict(test_df['jamo'][i])
    prob_list.append(pred[1][0])

100%|██████████| 1626/1626 [00:00<00:00, 27011.80it/s]


In [21]:
result_df = pd.DataFrame()
result_df['id'] = test_df['id']
result_df['smishing'] = prob_list

In [22]:
result_df.describe()

,id,smishing
count,1626.000000,1626.000000
mean,340812.500000,0.989630
std,469.530084,0.048730
min,340000.000000,0.508975
25%,340406.250000,0.999904
50%,340812.500000,1.000010
75%,341218.750000,1.000010
max,341625.000000,1.000010


In [13]:
pred = model.predict(test_df['jamo'][0])

print(pred[0][0], 'is the predicteed label')
print(pred[1][0], 'is the prob score')

__label__0 is the predicteed label
0.9999371767044067 is the prob score


In [14]:
pred = model.predict(test_df['jamo'][1])

print(pred[0][0], 'is the predicteed label')
print(pred[1][0], 'is the prob score')

__label__0 is the predicteed label
0.9846146702766418 is the prob score


In [16]:
pred = model.predict(test_df['jamo'][161])

print(pred[0][0], 'is the predicteed label')
print(pred[1][0], 'is the prob score')

__label__0 is the predicteed label
1.000009536743164 is the prob score


In [19]:
%%time
with open("../KB_NLP/jamo_label_test.txt",'w') as f:
    for i in range(len(test_df)): #
        #f.write("__label__")
        #f.write("{}".format(train_df['smishing'][i]))
        f.write("{}".format(test_df['jamo'][i]))
        #f.write("\n")

print("Make Labeled jamo")

Make Labeled jamo
CPU times: user 17.6 ms, sys: 3.92 ms, total: 21.6 ms
Wall time: 21.1 ms


In [20]:
f = open('../KB_NLP/jamo_label_test.txt','r')
test_text = f.readlines()
f.close()

In [39]:
pred = model.predict("ㄱㄴㄷㅇㄻㄴㅇㄻㄷㅈㅂㅈㄷㅂㅈㄷㅈㅂ담ㄴㅇㄴㅇㅂㅈㄷ")

print(pred[0][0], 'is the predicteed label')
print(pred[1][0], 'is the prob score')

__label__0 is the predicteed label
1.0000100135803223 is the prob score


In [30]:
pred?

Type:        tuple
String form: ([['__label__0']], array([[0.99953866]]))
Length:      2
Docstring:  
tuple() -> empty tuple
tuple(iterable) -> tuple initialized from iterable's items

If the argument is a tuple, the return value is the same object.


In [32]:
pred[1][2]

IndexError: index 2 is out of bounds for axis 0 with size 1

---
### Evaluation

In [27]:
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

print(roc_auc_score(labels, pred_labels))

NameError: name 'test' is not defined

In [24]:
pred?

Type:        tuple
String form: ([['__label__0']], array([[0.99953866]]))
Length:      2
Docstring:  
tuple() -> empty tuple
tuple(iterable) -> tuple initialized from iterable's items

If the argument is a tuple, the return value is the same object.


In [6]:
import fasttext

In [8]:
%%time
model = fasttext.train_supervised('../KB_NLP/jamo_corpus.txt', 
                                    #model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 2
                                    
                                )

ValueError: vector::_M_default_append

In [ ]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 2
                                    
                                )

In [ ]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 2
                                    
                                )

In [ ]:
model.save_model("../KB_NLP/bigram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")

In [ ]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 1
                                    
                                )

In [ ]:
model.save_model("../KB_NLP/unigram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")

In [ ]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 3
                                    
                                )

In [ ]:
model.save_model("../KB_NLP/trigram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")

In [ ]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 4
                                    
                                )

In [ ]:
model.save_model("../KB_NLP/4gram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")

In [ ]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 5
                                    
                                )

In [ ]:
model.save_model("../KB_NLP/5gram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")